In [1]:
import sys
import numpy as np
from ase import io

sys.path.insert(0, '../lib')
from ralign import options, remap, align

print(remap.__doc__)
print(align.__doc__)

nrecord,maplist,mapcount = remap(znums0,znums1,types0,types1,weights0,weights1,coords0,coords1,maxrecord,[natom0,natom1])

Wrapper for ``remap``.

Parameters
----------
znums0 : input rank-1 array('i') with bounds (natom0)
znums1 : input rank-1 array('i') with bounds (natom1)
types0 : input rank-1 array('i') with bounds (natom0)
types1 : input rank-1 array('i') with bounds (natom1)
weights0 : input rank-1 array('d') with bounds (natom0)
weights1 : input rank-1 array('d') with bounds (natom1)
coords0 : input rank-2 array('d') with bounds (3,natom0)
coords1 : input rank-2 array('d') with bounds (3,natom1)
maxrecord : input int

Other Parameters
----------------
natom0 : input int, optional
    Default: shape(znums0, 0)
natom1 : input int, optional
    Default: shape(znums1, 0)

Returns
-------
nrecord : int
maplist : rank-2 array('i') with bounds (natom0,maxrecord)
mapcount : rank-1 array('i') with bounds (maxrecord)

travec,rotmat = align(znums0,znums1,types0,types1,weights0,weights1,co

In [7]:
def optstr(string):
    return string + ' '*(options.optlen - len(string))

atoms0 = io.read('r005/100cobalt_j5.xyz', index=0)
atoms1 = io.read('r005/100cobalt_j5.xyz', index=1)

znums0 = atoms0.get_atomic_numbers()
znums1 = atoms1.get_atomic_numbers()
types0 = np.ones(len(atoms0), dtype=int)
types1 = np.ones(len(atoms1), dtype=int)
weights0 = np.ones(len(atoms0), dtype=float)
weights1 = np.ones(len(atoms1), dtype=float)
coords0 = np.transpose(atoms0.get_positions())
coords1 = np.transpose(atoms1.get_positions())

# Normalize weights
weights0 = weights0/sum(weights0)
weights1 = weights1/sum(weights1)

options.biased = True
options.converged = True
options.iterative = True
options.testing = True
options.mincount = 10
options.lenscale = 1000.0
options.tolerance = 0.17

In [13]:
n, maplist, mapcount = remap(znums0, znums1, types0, types1, weights0, weights1, coords0, coords1, 1)

 Map   Trial   Count   Cycles   Meanrot   Totalrot      RMSD
------------------------------------------------------------
   1       1      10      1.9      57.2     104.3     0.0482
------------------------------------------------------------
Found more than 1 mapping(s) in 12 random trial(s)


In [8]:
maplist[:, 0:n]

array([[ 35,  96],
       [ 68,  68],
       [ 27,  45],
       [ 31,  72],
       [ 24,  62],
       [ 70,  16],
       [ 25,  25],
       [ 75,  75],
       [  4,  74],
       [ 41,  26],
       [ 94, 100],
       [ 71,  71],
       [ 84,  61],
       [ 47,  47],
       [ 83,  89],
       [ 37,  30],
       [ 32,  32],
       [ 76,  88],
       [ 67,  49],
       [ 30,  37],
       [ 53,  77],
       [ 15,  69],
       [ 66,  66],
       [ 63,  63],
       [ 14,  54],
       [ 69,  15],
       [ 91,  59],
       [ 45,  27],
       [ 62,  24],
       [ 12,   1],
       [ 96,  35],
       [ 44,  48],
       [  5,   5],
       [  7,   8],
       [ 26,  41],
       [ 11,  11],
       [ 49,  67],
       [ 80,  93],
       [ 33,  98],
       [ 78,  78],
       [ 10,  82],
       [ 55,  21],
       [ 72,  31],
       [ 93,  80],
       [ 92,  20],
       [ 97,  22],
       [ 46,  46],
       [ 19,  18],
       [ 81,  81],
       [ 98,  33],
       [ 59,  91],
       [ 87,   6],
       [ 79,

In [7]:
mapcount[0:n]

array([10,  3], dtype=int32)